# Cascade Training for Viola-Jones Face Detector\n\nTrain 2-stage cascade classifier to reduce false positives.\n\n**V1 Configuration**:\n- Stage 1: T=10 (reject ~50% non-faces, keep >99% faces)\n- Stage 2: T=40 (final discriminative stage)\n- Combined: High detection rate with low FP rate\n\n**AI Usage**: Notebook structure assisted by Claude Code

In [ ]:
import numpy as np\nimport matplotlib.pyplot as plt\nimport pickle\nimport sys\nfrom pathlib import Path\n\n# Add src to path\nsys.path.insert(0, str(Path.cwd().parent / 'src'))\n\nfrom classifiers.cascade import train_cascade, evaluate_cascade, CascadeClassifier\nfrom features.haar_features import generate_haar_features\n\nplt.style.use('seaborn-v0_8-darkgrid')\n%matplotlib inline

## 1. Load Pre-computed Data

In [ ]:
print("Loading dataset labels...")\n\n# Load training data\nwith open('../data/processed/train_faces.pkl', 'rb') as f:\n    train_faces = pickle.load(f)\n\nwith open('../data/processed/train_nonfaces.pkl', 'rb') as f:\n    train_nonfaces = pickle.load(f)\n\n# Create labels\ny_train = np.concatenate([\n    np.ones(len(train_faces), dtype=int),\n    np.zeros(len(train_nonfaces), dtype=int)\n])\n\n# Load testing data\nwith open('../data/processed/test_faces.pkl', 'rb') as f:\n    test_faces = pickle.load(f)\n\nwith open('../data/processed/test_nonfaces.pkl', 'rb') as f:\n    test_nonfaces = pickle.load(f)\n\ny_test = np.concatenate([\n    np.ones(len(test_faces), dtype=int),\n    np.zeros(len(test_nonfaces), dtype=int)\n])\n\nprint(f"Training: {len(train_faces)} faces, {len(train_nonfaces)} non-faces")\nprint(f"Testing: {len(test_faces)} faces, {len(test_nonfaces)} non-faces")

In [ ]:
# Load pre-computed feature responses\nprint("\nLoading pre-computed feature responses...")\ntrain_responses = np.load('../data/processed/train_responses_10k.npy')\ntest_responses = np.load('../data/processed/test_responses_10k.npy')\n\nprint(f"Training responses: {train_responses.shape}")\nprint(f"Testing responses: {test_responses.shape}")

In [ ]:
# Load features\nprint("\nGenerating Haar features (same as training)...")\nfeatures = generate_haar_features(window_size=16, max_features=10000)\nprint(f"Generated {len(features)} features")

## 2. Define Cascade Configuration\n\n**2-Stage Cascade**:\n- Stage 1: Fast filter (T=10, high recall)\n- Stage 2: Discriminative (T=40, balanced)

In [ ]:
stage_configs = [\n    {\n        'T': 10,\n        'target_fpr': 0.5,    # Reject 50% of non-faces\n        'target_tpr': 0.995   # Keep 99.5% of faces\n    },\n    {\n        'T': 40,\n        'target_fpr': 0.01,   # Very low FP rate for final stage\n        'target_tpr': 0.99    # 99% face detection\n    }\n]\n\nprint("Cascade Configuration:")\nfor i, config in enumerate(stage_configs, 1):\n    print(f"  Stage {i}: T={config['T']}, FPR={config['target_fpr']:.1%}, TPR={config['target_tpr']:.1%}")

## 3. Train Cascade\n\n**This will take several minutes**\n- Stage 1: Train on full dataset\n- Stage 2: Train on faces + FPs from Stage 1

In [ ]:
cascade = train_cascade(\n    train_responses,\n    y_train,\n    features,\n    stage_configs,\n    verbose=True\n)

## 4. Evaluate on Training Set

In [ ]:
print("\nEvaluating cascade on TRAINING set...")\ntrain_metrics = evaluate_cascade(cascade, train_responses, y_train, verbose=True)

## 5. Evaluate on Test Set

In [ ]:
print("\nEvaluating cascade on TEST set...")\ntest_metrics = evaluate_cascade(cascade, test_responses, y_test, verbose=True)

## 6. Save Cascade Model

In [ ]:
model_path = Path('../data/models/cascade_v1_2stage.pkl')\ncascade.save(model_path)\nprint(f"\nCascade saved to {model_path}")

## 7. Compare with Single AdaBoost

In [ ]:
# Load single AdaBoost model if it exists\nfrom classifiers.adaboost import AdaBoostClassifier, evaluate_classifier\n\nadaboost_path = Path('../data/models/adaboost_v1_T50.pkl')\nif adaboost_path.exists():\n    print("Loading single AdaBoost model for comparison...")\n    adaboost = AdaBoostClassifier.load(adaboost_path)\n    \n    print("\nSingle AdaBoost (T=50) on TEST set:")\n    adaboost_metrics = evaluate_classifier(adaboost, test_responses, y_test, verbose=True)\n    \n    # Comparison table\n    print("\n" + "=" * 60)\n    print("COMPARISON: Cascade vs Single AdaBoost")\n    print("=" * 60)\n    print(f"{'Metric':<15} {'Cascade':<15} {'AdaBoost T=50':<15}")\n    print("-" * 60)\n    print(f"{'Accuracy':<15} {test_metrics['accuracy']:<15.2%} {adaboost_metrics['accuracy']:<15.2%}")\n    print(f"{'Precision':<15} {test_metrics['precision']:<15.2%} {adaboost_metrics['precision']:<15.2%}")\n    print(f"{'Recall':<15} {test_metrics['recall']:<15.2%} {adaboost_metrics['recall']:<15.2%}")\n    print(f"{'F1 Score':<15} {test_metrics['f1']:<15.2%} {adaboost_metrics['f1']:<15.2%}")\n    print("=" * 60)\nelse:\n    print("Single AdaBoost model not found. Train it first using 03_adaboost_training.ipynb")

## 8. Visualize Cascade Performance

In [ ]:
# Stage-by-stage rejection rates\nstage_info = test_metrics['stage_info']\n\nfig, axes = plt.subplots(1, 2, figsize=(14, 5))\n\n# Bar plot: samples per stage\nstages = [f'Stage {i+1}' for i in range(len(stage_info['samples_per_stage']))]\nsamples = stage_info['samples_per_stage']\nrejections = stage_info['rejections_per_stage']\n\naxes[0].bar(stages, samples, alpha=0.7, label='Samples evaluated', color='blue')\naxes[0].bar(stages, rejections, alpha=0.7, label='Samples rejected', color='red')\naxes[0].set_ylabel('Number of Samples', fontsize=12)\naxes[0].set_title('Cascade Stage Processing', fontsize=13, fontweight='bold')\naxes[0].legend()\naxes[0].grid(True, alpha=0.3, axis='y')\n\n# Rejection rates\nrejection_rates = [r/s if s > 0 else 0 for r, s in zip(rejections, samples)]\naxes[1].bar(stages, rejection_rates, alpha=0.7, color='orange')\naxes[1].set_ylabel('Rejection Rate', fontsize=12)\naxes[1].set_title('Rejection Rate per Stage', fontsize=13, fontweight='bold')\naxes[1].set_ylim([0, 1])\naxes[1].grid(True, alpha=0.3, axis='y')\n\n# Add percentage labels\nfor i, (stage, rate) in enumerate(zip(stages, rejection_rates)):\n    axes[1].text(i, rate + 0.02, f'{rate:.1%}', ha='center', fontsize=11)\n\nplt.tight_layout()\nplt.savefig('../results/figures/cascade_performance.png', dpi=150, bbox_inches='tight')\nplt.show()

## 9. Results Summary

In [ ]:
print("=" * 60)\nprint("CASCADE V1 RESULTS SUMMARY")\nprint("=" * 60)\nprint(f"\nConfiguration:")\nprint(f"  Stages: {len(cascade.stages)}")\nprint(f"  Total weak classifiers: {sum(len(s.classifier.weak_classifiers) for s in cascade.stages)}")\nfor i, stage in enumerate(cascade.stages, 1):\n    print(f"    Stage {i}: {len(stage.classifier.weak_classifiers)} weak classifiers, threshold={stage.threshold:.4f}")\n\nprint(f"\nTest Performance:")\nprint(f"  Accuracy:  {test_metrics['accuracy']:.2%}")\nprint(f"  Precision: {test_metrics['precision']:.2%}")\nprint(f"  Recall:    {test_metrics['recall']:.2%}")\nprint(f"  F1 Score:  {test_metrics['f1']:.2%}")\n\nprint(f"\nComputational Efficiency:")\ntotal_samples = stage_info['samples_per_stage'][0]\nfor i, n_samples in enumerate(stage_info['samples_per_stage'], 1):\n    pct = n_samples / total_samples if total_samples > 0 else 0\n    print(f"  Stage {i}: {pct:.1%} of samples evaluated")\n\nprint("\n" + "=" * 60)\nif test_metrics['accuracy'] >= 0.70:\n    print("[OK] V1 Complete! Cascade achieves >70% accuracy")\nelse:\n    print("Note: Consider tuning thresholds or adding more stages")\nprint("=" * 60)

## Next Steps\n\n1. **V1 Milestone**: If accuracy >70%, Part 1 implementation complete!\n2. **V2 Scale-up**: Increase to 50k features, T=200, more cascade stages\n3. **Part 2**: Implement sliding window detection on full images\n4. **Report**: Document algorithm, results, and analysis